In [1]:
import time
from selenium import webdriver
import urllib
from urllib.request import Request,urlopen

## Scraping 
#### https://in.canon/en
Scraping all the Canon Lens present on gsmarena and storing them in Firebase database

In [2]:
driver = webdriver.Chrome('chromedriver')
driver.get('https://in.canon/en/consumer/products/search?category=photography&subCategory=ef-lenses')
time.sleep(5)

In [3]:
import requests
from bs4 import BeautifulSoup
import re

In [23]:
data=BeautifulSoup(driver.page_source,'html.parser')

In [6]:
all_lens_urls_list=[]
lens_nm_list = []
   
for getdivtag in data.findAll('div', {'class': 'post-item'}):
    for getimgtag in getdivtag.findAll('img',src=True):
        all_lens_urls_list.append(getimgtag['src'])

for gettitletag in data.findAll('h5',{'class': 'title'}):
    for gettitle in gettitletag.findAll(['a']):
        lens_nm_list.append(gettitle.text)


In [7]:
lens_nm_list
    

['\n                                                    Extender EF 1.4xIII\n                                                ',
 '\n                                                    EF180mm f/3.5L Macro USM\n                                                ',
 '\n                                                    TS-E50mm f/2.8L Macro\n                                                ',
 '\n                                                    TS-E24mm f/3.5L II\n                                                ',
 '\n                                                    TS-E17mm f/4L\n                                                ',
 '\n                                                    RF85mm f/1.2L USM DS\n                                                ',
 '\n                                                    RF85mm f/1.2L USM\n                                                ',
 '\n                                                    RF70-200mm f/2.8L IS USM\n                       

In [8]:
len(all_lens_urls_list)

30

### Single Value Testing

In [9]:
driver.get(all_lens_urls_list[0])
time.sleep(10)

In [10]:
img_nm = all_lens_urls_list[0].rsplit('/',1)[1].rsplit('.',1)
img_name = img_nm[0] + '.png'
img_name

'ef-14xiii-b1.png'

In [26]:
urllib.request.urlretrieve(all_lens_urls_list[0], img_name)

('ef-14xiii-b1.png', <http.client.HTTPMessage at 0xe9d050>)

In [42]:
for i in all_lens_urls_list:
    driver.get(i)
    time.sleep(10)
    img_nm = i.rsplit('/',1)[1].rsplit('.',1)
    img_name = img_nm[0] + '.png'
    urllib.request.urlretrieve(i, img_name)
    

In [43]:
img_name

'ef400mm-f4-do-is-ii-usm-b1.png'

### END

### Full Data Scrapping

In [14]:
temp_lens_list = []
new_lens_list = []
for i in range(0,len(lens_nm_list)):
    temp_lens_list.append(lens_nm_list[i].split('\n'))
    
for i in range(0,len(temp_lens_list)):
    for j in range(0,3):
        if(temp_lens_list[i][j] != ''):
            new_lens_list.append(temp_lens_list[i][j].strip())

In [15]:
final_lens_list =[]
for i in range(0,len(new_lens_list)):
    if(new_lens_list[i] != ''):
        final_lens_list.append(new_lens_list[i])

In [16]:
final_lens_list

['Extender EF 1.4xIII',
 'EF180mm f/3.5L Macro USM',
 'TS-E50mm f/2.8L Macro',
 'TS-E24mm f/3.5L II',
 'TS-E17mm f/4L',
 'RF85mm f/1.2L USM DS',
 'RF85mm f/1.2L USM',
 'RF70-200mm f/2.8L IS USM',
 'RF50mm f/1.2L USM',
 'RF35mm f/1.8 MACRO IS STM',
 'RF28-70mm f/2L USM',
 'RF24-70mm f/2.8L IS USM',
 'RF24-105mm f/4L IS USM',
 'RF15-35mm f/2.8L IS USM',
 'EF85mm f/1.2L II USM',
 'EF800mm f/5.6L IS USM',
 'EF8-15mm f/4L Fisheye USM',
 'EF75-300mm f/4-5.6 III USM',
 'EF70-300mm f/4-5.6L IS USM',
 'EF70-300mm f/4-5.6 IS II USM',
 'EF70-200mm f/4L USM',
 'EF70-200mm f/4L IS USM',
 'EF70-200mm f/2.8L IS III USM',
 'EF70-200mm f/2.8L IS II USM',
 'EF600mm f/4L IS III USM',
 'EF50mm f/1.4 USM',
 'EF500mm f/4L IS II USM',
 'EF40mm f/2.8 STM',
 'EF400mm f/5.6L USM',
 'EF400mm f/4 DO IS II USM']

In [17]:
title = final_lens_list[0]
title

'Extender EF 1.4xIII'

In [18]:
key = title.lower().replace(' ','-')
key

'extender-ef-1.4xiii'

### Firebase 

In [11]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Fetch the service account key JSON file contents
cred = credentials.Certificate('serviceAccountKey.json')

# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://personal-portfolio-12c66.firebaseio.com/'
})

In [12]:
# As an admin, the app has access to read and write all data, regradless of Security Rules
ref = db.reference('lens/canon/products')

### Final code

In [22]:
for i in range(0,len(all_lens_urls_list)):
    driver.get(all_lens_urls_list[i])
    time.sleep(10)
    
    title = final_lens_list[i]
    #title
    print(title)
    
    key = title.lower().replace(' ','-').replace('.','_').replace('/','')
    #key 
    
    img_name = final_lens_list[i].replace(' ','') + '.png'
    #img_name
    
    time.sleep(10)
    
    img_nm = all_lens_urls_list[i].rsplit('/',1)[1].rsplit('.',1)
    img_name = img_nm[0] + '.png'
    urllib.request.urlretrieve(all_lens_urls_list[i], img_name)

    ref.child(key).set({
        'p_img': "lenses/"+img_name,
        'p_name': title,
    })
    

Extender EF 1.4xIII
EF180mm f/3.5L Macro USM
TS-E50mm f/2.8L Macro
TS-E24mm f/3.5L II
TS-E17mm f/4L
RF85mm f/1.2L USM DS
RF85mm f/1.2L USM
RF70-200mm f/2.8L IS USM
RF50mm f/1.2L USM
RF35mm f/1.8 MACRO IS STM
RF28-70mm f/2L USM
RF24-70mm f/2.8L IS USM
RF24-105mm f/4L IS USM
RF15-35mm f/2.8L IS USM
EF85mm f/1.2L II USM
EF800mm f/5.6L IS USM
EF8-15mm f/4L Fisheye USM
EF75-300mm f/4-5.6 III USM
EF70-300mm f/4-5.6L IS USM
EF70-300mm f/4-5.6 IS II USM
EF70-200mm f/4L USM
EF70-200mm f/4L IS USM
EF70-200mm f/2.8L IS III USM
EF70-200mm f/2.8L IS II USM
EF600mm f/4L IS III USM
EF50mm f/1.4 USM
EF500mm f/4L IS II USM
EF40mm f/2.8 STM
EF400mm f/5.6L USM
EF400mm f/4 DO IS II USM
